# Imports

In [2]:
# For CSV files
import pandas as pd

# Data Exploration

In [6]:
#Import CSV file
stations_df = pd.read_csv("../Data/ov_haltes.csv", sep=";")

In [7]:
stations_df.head()

,OBJECTNUMMER,Modaliteit,Lijn,Lijn_select,RADIUS,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 9
0,1,Tram,2 | 11 | 12 | 13 | 17,02|11|12|13|17,3,"POINT(4.893349,52.376064)","POINT(52.376064,4.893349)","4,893349","52,376064",NaN
1,2,Tram,1 | 11 | 17,01|11|17,3,"POINT(4.853122,52.358377)","POINT(52.358377,4.853122)","4,853122","52,358377",NaN
2,3,Tram,1 | 17,01|17,3,"POINT(4.82733,52.357809)","POINT(52.357809,4.82733)","4,82733","52,357809",NaN
3,4,Tram,2 | 11 | 12,02|11|12,3,"POINT(4.889295,52.369254)","POINT(52.369254,4.889295)","4,889295","52,369254",NaN
4,5,Tram,2 | 11 | 12,02|11|12,3,"POINT(4.884471,52.365017)","POINT(52.365017,4.884471)","4,884471","52,365017",NaN
